# Assignment Part 1 - Data Scraping and Transformation

### The assignment includes scraping unstructured data available openly on Wikipedia and transform it into Structured format with cleaning up to be used for Analysis

#### Define and import pre-requisites for Python - Pandas and Data Scraping using BeautifulSoup

In [7]:
from bs4 import BeautifulSoup # Beautiful Soup Data scraping library
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # Pandas data analsysis library
import requests # HTTP library
from sklearn.cluster import KMeans # Data clustering algorithm


!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('All Libraries are imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

### Scraping Data from Open Website - Wikipedia.

#### scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes

In [12]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [13]:
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [16]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

#### List first 4 rows from imported data

In [17]:
df.head(4)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


### Data Cleaning to remove Blank or unassigned Borough

#### The "Not assigned" values will be replaced with the Borough name in that cell from the Neighborhood column

In [18]:
df=df[df['Borough']!='Not assigned']

df[df['Neighborhood']=='Not assigned']=df['Borough']
df.head(4)
      

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights


In [22]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

df_merge = pd.merge(df, temp_df, on='Postalcode')

df_merge.drop(['Neighborhood'],axis=1,inplace=True)

df_merge.drop_duplicates(inplace=True)

df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,Queen's Park,Queen's Park,Queen's Park


#### Print the number of rows of dataframe using ".shape" option

In [26]:
print('The DataFrame shape is', df_merge.shape)

The DataFrame shape is (103, 3)
